# COVTYPE

# IMPORTS

In [1]:
import numpy as np
import pandas as pd
import requests, zipfile, io

from sklearn.metrics import accuracy_score, f1_score, auc, roc_auc_score, roc_curve, make_scorer
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


# Algo 1: LOGISTIC REGRESSION


In [2]:
def log_reg_classifier(X_train, Y_train, X_test, Y_test):
    # define hyperparameters to search through & error metrics 
    C_list = [10**-8,10**-7,10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,10**0,10**1,10**2,10**3,10**4]
    penalty_list = ['l2', 'l1', 'none']
    solver = ['saga'] #this solver works for no penalty, l1 and l2 penalties
    scoring = {'Accuracy': make_scorer(accuracy_score), 'f1': 'f1', 'AUC': 'roc_auc'}
    classifier = LogisticRegression()
        
    # gridsearch 
    lr =  GridSearchCV(classifier, {'C':C_list, 'penalty':penalty_list,'solver':solver},
                                   scoring=scoring,refit=False, cv=5, n_jobs=-1, 
                                   return_train_score=True, verbose=2 ) 
        
    lr.fit(X_train,Y_train)
    results = pd.DataFrame(lr.cv_results_)
        
    # Get each parameter settings that gives best accuracy, F1, and AUC on validation set
    best_AUC = results[results['rank_test_AUC']==1]['params'].values[0]
    best_accuracy = results[results['rank_test_Accuracy']  ==1]['params'].values[0]
    best_F1 = results[results['rank_test_f1']  ==1]['params'].values[0]
        
    # Train 3 models using the 5000 samples and each of the 3 best parameter settings (one model per metric)
    clf_AUC = LogisticRegression(C=best_AUC['C'], solver = best_AUC['solver'],
                             penalty=best_AUC['penalty']).fit(X_train, Y_train)

    clf_accuracy = LogisticRegression(C=best_accuracy['C'], solver = best_accuracy['solver'],
                             penalty=best_accuracy['penalty']).fit(X_train, Y_train)

    clf_F1 = LogisticRegression(C=best_F1['C'], solver = best_F1['solver'],
                                penalty=best_F1['penalty']).fit(X_train, Y_train)
        
    # For average training performance
    train_accuracy = clf_accuracy.score(X_train, Y_train)  # Accuracy  
    train_F1 = f1_score(Y_train, clf_F1.predict(X_train))   # F1
    fpr, tpr, threshold = roc_curve(Y_train, clf_AUC.predict_proba(X_train)[:,1])
    train_auc = auc(fpr, tpr) # AUC
        
    # Find and store accuracy, F1, and AUC of the 3 models from previous line on test set
    test_accuracy = clf_accuracy.score(X_test, Y_test)  # Accuracy
    
    test_F1 = f1_score(Y_test, clf_F1.predict(X_test))   # F1
    
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, clf_AUC.predict_proba(X_test)[:,1])
    test_auc = auc(false_positive_rate, true_positive_rate) # AUC

    return train_accuracy, train_F1, train_auc, test_accuracy, test_F1, test_auc


# Algo 2: RANDOM FOREST

In [3]:
def rf_classifier(X_train,Y_train,X_test,Y_test):
    # define hyperparameters to search through & error metrics  
    n_estimators = [1024]
    min_samples_split = [1,2,4,6,8,12,16,20]
    scoring = {'Accuracy': make_scorer(accuracy_score), 'f1': 'f1', 'AUC': 'roc_auc'}
    classifier = RandomForestClassifier()

    rf =  GridSearchCV(classifier, {'n_estimators':n_estimators, 'min_samples_split':min_samples_split},
                                   scoring=scoring,refit=False, cv=5, n_jobs=-1, 
                                   return_train_score=True, verbose=2) 

    rf.fit(X_train,Y_train)
    results = pd.DataFrame(rf.cv_results_)
        
    # Get each parameter settings that gives best accuracy, F1, and AUC on validation set
    best_AUC = results[results['rank_test_AUC']==1]['params'].values[0]
    best_accuracy = results[results['rank_test_Accuracy']  ==1]['params'].values[0]
    best_F1 = results[results['rank_test_f1']  ==1]['params'].values[0]

    # Train 3 models using the 5000 samples and each of the 3 best parameter settings (one model per metric)
    clf_AUC = RandomForestClassifier(n_estimators=best_AUC['n_estimators'], 
                              min_samples_split=best_AUC['min_samples_split']).fit(X_train, Y_train)

    clf_accuracy = RandomForestClassifier(n_estimators=best_accuracy['n_estimators'], 
                              min_samples_split=best_accuracy['min_samples_split']).fit(X_train, Y_train)

    clf_F1 = RandomForestClassifier(n_estimators=best_F1['n_estimators'], 
                                  min_samples_split=best_F1['min_samples_split']).fit(X_train, Y_train)

    # For average training performance
    train_accuracy = clf_accuracy.score(X_train, Y_train)  # Accuracy  
    train_F1 = f1_score(Y_train, clf_F1.predict(X_train))   # F1
    fpr, tpr, threshold = roc_curve(Y_train, clf_AUC.predict_proba(X_train)[:,1])
    train_auc = auc(fpr, tpr) # AUC
        
    # Find and store accuracy, F1, and AUC of the 3 models from previous line on test set
    test_accuracy = clf_accuracy.score(X_test, Y_test)  # Accuracy
    
    test_F1 = f1_score(Y_test, clf_F1.predict(X_test))   # F1
    
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, clf_AUC.predict_proba(X_test)[:,1])
    test_auc = auc(false_positive_rate, true_positive_rate) # AUC

    return train_accuracy, train_F1, train_auc, test_accuracy, test_F1, test_auc
  

# Algo 3: KNN

In [4]:
def knn_classifier(X_train,Y_train,X_test,Y_test):
    # define hyperparameters to search through & error metrics  
    n_neighbors = np.linspace(1, 105, num=27, dtype=int)
    weights = ['uniform', 'distance']
    p = [1,2]
    scoring = {'Accuracy': make_scorer(accuracy_score), 'f1': 'f1', 'AUC': 'roc_auc'}
    classifier = KNeighborsClassifier()

    rf =  GridSearchCV(classifier, {'n_neighbors':n_neighbors, 'weights': weights, 'p':p},
                                   scoring=scoring,refit=False, cv=5, n_jobs=-1, 
                                   return_train_score=True, verbose=2) 

    rf.fit(X_train,Y_train)
    results = pd.DataFrame(rf.cv_results_)
        
    # Get each parameter settings that gives best accuracy, F1, and AUC on validation set
    best_AUC = results[results['rank_test_AUC']==1]['params'].values[0]
    best_accuracy = results[results['rank_test_Accuracy']  ==1]['params'].values[0]
    best_F1 = results[results['rank_test_f1']  ==1]['params'].values[0]

    # Train 3 models using the 5000 samples and each of the 3 best parameter settings (one model per metric)
    clf_AUC = KNeighborsClassifier(n_neighbors=best_AUC['n_neighbors'], 
                             weights=best_AUC['weights'], p = best_AUC['p']).fit(X_train, Y_train)

    clf_accuracy = KNeighborsClassifier(n_neighbors=best_accuracy['n_neighbors'], 
                              weights=best_accuracy['weights']).fit(X_train, Y_train)

    clf_F1 = KNeighborsClassifier(n_neighbors=best_F1['n_neighbors'], 
                                  weights=best_F1['weights'], p = best_F1['p']).fit(X_train, Y_train)

    # For average training performance
    train_accuracy = clf_accuracy.score(X_train, Y_train)  # Accuracy  
    train_F1 = f1_score(Y_train, clf_F1.predict(X_train))   # F1
    fpr, tpr, threshold = roc_curve(Y_train, clf_AUC.predict_proba(X_train)[:,1])
    train_auc = auc(fpr, tpr) # AUC
        
    # Find and store accuracy, F1, and AUC of the 3 models from previous line on test set
    test_accuracy = clf_accuracy.score(X_test, Y_test)  # Accuracy
    
    test_F1 = f1_score(Y_test, clf_F1.predict(X_test))   # F1
    
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, clf_AUC.predict_proba(X_test)[:,1])
    test_auc = auc(false_positive_rate, true_positive_rate) # AUC

    return train_accuracy, train_F1, train_auc, test_accuracy, test_F1, test_auc  

# DATA - COVTYPE

In [5]:
data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz",
                    header=None)

In [6]:
rows_to_drop = data[data.isnull().any(axis=1)] 
print(rows_to_drop)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]
Index: []

[0 rows x 55 columns]


In [7]:
data[54].value_counts() #looking for largest class to make as the positive class

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: 54, dtype: int64

In [8]:
# get inputs (X) & targets (Y) 
X = data.loc[: , 0:53] #feature inputs: taking colums "0" to "53"
labels = data[54]

#determining +/- classes, + class is Lodgepole Pine (2), - class is everything else
Y=[]
for i in labels:
    if i == 2:
        Y += [1]
    else:
        Y += [-1]
        
data['y'] = Y
data

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,y
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,5,-1
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,5,-1
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,2,1
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,2,1
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,5,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,3,-1
581008,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,3,-1
581009,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,3,-1
581010,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,3,-1


In [9]:
# %POZ
print(data.shape)
print(data['y'].value_counts())

percent_positive = 283301/581012
print(percent_positive)

(581012, 56)
-1    297711
 1    283301
Name: y, dtype: int64
0.48759922342395684


### 5 Trials across the three algos 

In [10]:
log_reg_data = []
rf_data = []
knn_data = []

for i in range(5):
    #split data for training and testing
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, train_size=5000)

        
    #tansform data for Logistic Regression and KNN
    X_train_t = StandardScaler().fit_transform(X_train)    
    X_test_t = StandardScaler().fit_transform(X_test)
    
    log_reg_data += log_reg_classifier(X_train_t, Y_train, X_test_t, Y_test)
    rf_data += rf_classifier(X_train,Y_train, X_test, Y_test)
    knn_data += knn_classifier(X_train_t,Y_train, X_test_t, Y_test)
    
    

Fitting 5 folds for each of 39 candidates, totalling 195 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done 195 out of 195 | elapsed:   20.2s finished
/Users/annaleaohalloran/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/annaleaohalloran/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/annaleaohalloran/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/Users/annaleaohalloran/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  2.1min finished


Fitting 5 folds for each of 108 candidates, totalling 540 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 195 out of 195 | elapsed:   16.4s finished
/Users/annaleaohalloran/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/annaleaohalloran/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/annaleaohalloran/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.9min finished


Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   35.8s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  4.0min finished


In [11]:
#Logistic Regression Results
df_log_reg = pd.DataFrame(columns=('Train ACC', 'Train F1','Train AUC', 'Test ACC', 'Test F1', 'Test AUC'))
df_log_reg.loc['Trial 1'] = log_reg_data[0:6]
df_log_reg.loc['Trial 2'] = log_reg_data[6:12]
df_log_reg.loc['Trial 3'] = log_reg_data[12:18]
df_log_reg.loc['Trial 4'] = log_reg_data[18:24]
df_log_reg.loc['Trial 5'] = log_reg_data[24:32]

#Random Forest Results
df_rf = pd.DataFrame(columns=('Train ACC', 'Train F1','Train AUC', 'Test ACC', 'Test F1', 'Test AUC'))
df_rf.loc['Trial 1'] = rf_data[0:6]
df_rf.loc['Trial 2'] = rf_data[6:12]
df_rf.loc['Trial 3'] = rf_data[12:18]
df_rf.loc['Trial 4'] = rf_data[18:24]
df_rf.loc['Trial 5'] = rf_data[24:32]


#KNN Results
df_knn = pd.DataFrame(columns=('Train ACC', 'Train F1','Train AUC', 'Test ACC', 'Test F1', 'Test AUC'))
df_knn.loc['Trial 1'] = knn_data[0:6]
df_knn.loc['Trial 2'] = knn_data[6:12]
df_knn.loc['Trial 3'] = knn_data[12:18]
df_knn.loc['Trial 4'] = knn_data[18:24]
df_knn.loc['Trial 5'] = knn_data[24:32]

print('Logistic Regression Results')
display(df_log_reg)
print(df_log_reg.mean())

print('\n\nRandom Forest Results')
display(df_rf)
print(df_rf.mean())

print('\n\nKNN Results')
display(df_knn)
print(df_knn.mean())


Logistic Regression Results


,Train ACC,Train F1,Train AUC,Test ACC,Test F1,Test AUC
Trial 1,0.7566,0.756627,0.829217,0.512420,0.0,0.5
Trial 2,0.7558,0.744721,0.830290,0.512279,0.0,0.5
Trial 3,0.7604,0.758857,0.830281,0.512375,0.0,0.5
Trial 4,0.7532,0.747235,0.823051,0.512361,0.0,0.5
Trial 5,0.7362,0.729990,0.812912,0.512363,0.0,0.5


Train ACC    0.752440
Train F1     0.747486
Train AUC    0.825150
Test ACC     0.512359
Test F1      0.000000
Test AUC     0.500000
dtype: float64


Random Forest Results


,Train ACC,Train F1,Train AUC,Test ACC,Test F1,Test AUC
Trial 1,1.0,1.0,1.0,0.822443,0.820710,0.902272
Trial 2,1.0,1.0,1.0,0.817943,0.816475,0.899476
Trial 3,1.0,1.0,1.0,0.820754,0.817227,0.899197
Trial 4,1.0,1.0,1.0,0.825196,0.823592,0.903400
Trial 5,1.0,1.0,1.0,0.825901,0.823708,0.904316


Train ACC    1.000000
Train F1     1.000000
Train AUC    1.000000
Test ACC     0.822447
Test F1      0.820342
Test AUC     0.901732
dtype: float64


KNN Results


,Train ACC,Train F1,Train AUC,Test ACC,Test F1,Test AUC
Trial 1,1.0,1.0,1.0,0.781459,0.787115,0.871582
Trial 2,1.0,1.0,1.0,0.780466,0.788280,0.871782
Trial 3,1.0,1.0,1.0,0.782027,0.790807,0.873219
Trial 4,1.0,1.0,1.0,0.779503,0.788297,0.872265
Trial 5,1.0,1.0,1.0,0.775331,0.785086,0.863671


Train ACC    1.000000
Train F1     1.000000
Train AUC    1.000000
Test ACC     0.779757
Test F1      0.787917
Test AUC     0.870503
dtype: float64
